In [77]:
import pandas as pd

## 데이터 불러오기

In [78]:
df = pd.read_csv('data/신한카드/신한카드_최종.txt', sep='|', encoding='cp949', dtype={'LALXCVL':str, 'LALYCVL':str})

In [8]:
df = pd.read_csv('data/신한카드/신한카드_alt_processing.csv', encoding='UTF8')

C:\Users\Puser\AppData\Local\Temp\ipykernel_13292\1214190466.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/신한카드/신한카드_alt_processing.csv', encoding='UTF8')


In [79]:
df

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
0,202304,양식,큐리,NaN,20230316,이태원,126.989621982174,37.533447966088,23549507,450,...,37.818,8.495,2.765,5.506,83.458,6.029,5.805,서울특별시 용산구 녹사평대로 168-11,양식,NaN
1,202307,한식,온달네,NaN,20130329,NaN,127.075785890574,37.630286318763,67329528,978,...,40.950,20.372,11.218,11.734,40.915,4.749,32.381,"서울 노원구 공릉로 235,",한식,한식
2,202312,한식,연희동 만석집,NaN,20190812,연희동,126.929970755789,37.566635948945,19803263,136,...,12.267,10.279,16.920,53.722,18.824,10.195,68.443,서울특별시 서대문구 연희로 93-7,한식,한식
3,202312,커피전문점,숲(Soop),NaN,20211027,NaN,127.001453656784,37.61763382133,13708933,1211,...,35.152,24.502,15.805,9.296,34.800,2.890,15.467,서울특별시 성북구 보국문로 180,음료,NaN
4,202312,한식,무침표홍어무침&보쌈,NaN,20230823,신월4동,126.843492321048,37.516762611414,7486687,249,...,44.074,13.833,27.153,9.014,30.869,31.842,27.747,서울특별시 양천구 남부순환로83길 47,한식,한식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1386417,202311,중식,동순루,NaN,20060425,NaN,127.025290571682,37.566960355172,7374391,380,...,34.808,26.637,11.467,14.357,9.526,8.126,30.519,서울 성동구 하왕십리1동,아시아음식,중식
1386418,202308,한식,돈선생 방배점,NaN,20230117,방배역,126.995642877144,37.483527018776,13942370,206,...,24.979,26.070,6.140,6.809,44.212,0.000,45.183,서울특별시 서초구 효령로31길 52,한식,NaN
1386419,202307,일반대중음식,붕어앤호두커피,NaN,20211111,어린이대공원,127.092478736356,37.551565391221,520752,140,...,47.715,14.381,8.361,0.000,53.958,4.236,7.246,서울특별시 광진구 자양로48길 6,음료,다방/커피숍/카페
1386420,202305,한식,큰손곱창,NaN,20140207,자양,127.081161663684,37.533096400933,3526346,238,...,43.646,27.430,10.636,0.000,16.981,1.306,31.704,"서울 광진구 뚝섬로55길 45,",한식,한식


In [88]:
# 위도 또는 경도가 '0'인 경우 주소별로 처리하기 위해 조건 함수 정의
def group_key(row):
    if row['LALXCVL'] == '0' or row['LALYCVL'] == '0':
        return (row['MCTNM'], row['MCT_BSE_AR'])
    else:
        return (row['MCTNM'], row['LALXCVL'], row['LALYCVL'])

# 데이터에 group_key 함수 적용하여 그룹화 키 생성
df['group_key'] = df.apply(group_key, axis=1)

# 결과를 저장할 데이터프레임 초기화
filled_df_corrected = pd.DataFrame()

# group_key 별로 처리
for _, group_data in df.groupby('group_key'):
    group_data_sorted = group_data.sort_values(by='TA_YM')
    existing_months = group_data_sorted['TA_YM'].tolist()
    
    # 비연속적인 연월 찾기
    for i, month in enumerate(existing_months[:-1]):
        next_month = existing_months[i + 1]
        if next_month - month > 1:
            for missing_month in range(month + 1, next_month):
                missing_data = group_data_sorted.iloc[0].copy()
                missing_data['TA_YM'] = missing_month
                
                # 앞뒤 연월의 EST_AMT_TOT 값의 평균 계산
                prev_month_amt = group_data_sorted[group_data_sorted['TA_YM'] == month]['EST_AMT_TOT'].values[0]
                next_month_amt = group_data_sorted[group_data_sorted['TA_YM'] == next_month]['EST_AMT_TOT'].values[0]
                avg_amt = (prev_month_amt + next_month_amt) / 2
                
                # 컬럼 값 설정
                for col in missing_data.index:
                    if col not in ['TA_YM', 'MCT_RY_NM', 'MCTNM', 'BRD_NM', 'ARED', 'BZNCD', 'LALXCVL', 'LALYCVL', 'EST_AMT_TOT', 'MCT_BSE_AR', 'MCT_RY_NM2', 'OUB_RY_MI_CZ_NM', 'group_key']:
                        missing_data[col] = 0
                missing_data['EST_AMT_TOT'] = avg_amt
                
                filled_df_corrected = pd.concat([filled_df_corrected, pd.DataFrame([missing_data])], ignore_index=True)

# group_key 컬럼 제거
df.drop(columns='group_key', inplace=True)

# 기존 데이터와 수정된 누락된 데이터를 합치기
result_df = pd.concat([df, filled_df_corrected], ignore_index=True).sort_values(by=['MCTNM', 'LALXCVL', 'LALYCVL', 'TA_YM'])
result_df.drop(columns='group_key', inplace=True)
result_df

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
718779,202301,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,32082497.0,442,...,41.902,20.308,13.501,4.576,68.725,2.912,17.340,서울 종로구 돈화문로11가길 7,한식,한식
163056,202302,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,51156355.0,704,...,34.331,28.429,9.758,7.113,60.367,6.643,20.998,서울 종로구 돈화문로11가길 7,한식,한식
884033,202303,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,54534440.0,733,...,37.938,19.830,11.116,12.763,45.212,7.570,28.394,서울 종로구 돈화문로11가길 7,한식,한식
584501,202304,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,84513865.0,1015,...,43.340,13.873,10.164,12.144,57.957,9.534,15.989,서울 종로구 돈화문로11가길 7,한식,한식
50193,202305,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,70911682.0,850,...,38.152,21.813,12.378,7.416,58.094,5.134,16.537,서울 종로구 돈화문로11가길 7,한식,한식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380521,202308,양식,（주）에스알인베,NaN,20100121,수서역,127.103226986877,37.488510452832,49810547.0,1128,...,22.435,19.344,18.120,28.904,24.578,11.407,39.024,"서울 강남구 광평로51길 6-11,",양식,NaN
129651,202309,양식,（주）에스알인베,NaN,20100121,수서역,127.103226986877,37.488510452832,40096531.0,1015,...,27.881,19.919,12.677,29.676,28.544,13.324,36.674,"서울 강남구 광평로51길 6-11,",양식,NaN
344382,202310,양식,（주）에스알인베,NaN,20100121,수서역,127.103226986877,37.488510452832,33497913.0,874,...,28.690,22.706,17.848,23.248,32.342,16.282,26.179,"서울 강남구 광평로51길 6-11,",양식,NaN
1147106,202311,양식,（주）에스알인베,NaN,20100121,수서역,127.103226986877,37.488510452832,29845619.0,853,...,29.292,14.482,12.809,27.874,34.191,20.221,24.117,"서울 강남구 광평로51길 6-11,",양식,NaN


In [89]:
result_df[result_df['MCTNM']=='GS25삼육병원점']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
884517,202303,편의점,GS25삼육병원점,NaN,20120201,NaN,0,0,159214.0,16,...,49.627,19.729,0.0,0.0,80.271,0.0,19.729,서울 동대문구 망우로16길,기타,소매
1388625,202304,편의점,GS25삼육병원점,NaN,20120201,NaN,0,0,150307.0,0,...,0.000,0.000,0.0,0.0,0.000,0.0,0.000,서울 동대문구 망우로16길,기타,소매
1388626,202305,편의점,GS25삼육병원점,NaN,20120201,NaN,0,0,150307.0,0,...,0.000,0.000,0.0,0.0,0.000,0.0,0.000,서울 동대문구 망우로16길,기타,소매
1388627,202306,편의점,GS25삼육병원점,NaN,20120201,NaN,0,0,150307.0,0,...,0.000,0.000,0.0,0.0,0.000,0.0,0.000,서울 동대문구 망우로16길,기타,소매
299853,202307,편의점,GS25삼육병원점,NaN,20120201,NaN,0,0,141400.0,17,...,50.078,0.000,0.0,0.0,100.000,0.0,0.000,서울 동대문구 망우로16길,기타,소매


In [90]:
# 위도 또는 경도가 '0'인 경우 주소별로 처리하기 위한 조건 함수 정의
def group_key(row):
    if row['LALXCVL'] == '0' or row['LALYCVL'] == '0':
        return (row['MCTNM'], row['MCT_BSE_AR'], row['TA_YM'])
    else:
        return (row['MCTNM'], row['LALXCVL'], row['LALYCVL'], row['TA_YM'])

# 데이터에 group_key 함수 적용하여 그룹화 키 생성
result_df['group_key'] = result_df.apply(group_key, axis=1)

# 그룹별로 EST_AMT_TOT 합계 계산 및 첫 번째 행 데이터로 나머지 컬럼 채우기
aggregated_data = result_df.groupby('group_key').agg({
    'EST_AMT_TOT': 'sum'  # 매출액 합계
}).reset_index()

# 나머지 컬럼을 위해 첫 번째 행을 가져오기
first_row_data = result_df.groupby('group_key').first().reset_index()

# 합계된 매출액과 첫 번째 행 데이터 병합
merged_data = pd.merge(aggregated_data, first_row_data, on='group_key', suffixes=('_sum', ''))

# 불필요한 컬럼 제거 및 원래 컬럼 순서로 재정렬
final_df = merged_data.drop(columns=['EST_AMT_TOT_sum']).drop_duplicates(subset='group_key').sort_values(by=['MCTNM', 'LALXCVL', 'LALYCVL', 'TA_YM'])
final_df.drop(columns=['group_key'], inplace=True)

# 결과 확인
final_df

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
0,202301,한식,"""미 ""갈매기살전문",None,20031105,익선동,126.991068861385,37.573007173395,32082497.0,442,...,41.902,20.308,13.501,4.576,68.725,2.912,17.340,서울 종로구 돈화문로11가길 7,한식,한식
1,202302,한식,"""미 ""갈매기살전문",None,20031105,익선동,126.991068861385,37.573007173395,51156355.0,704,...,34.331,28.429,9.758,7.113,60.367,6.643,20.998,서울 종로구 돈화문로11가길 7,한식,한식
2,202303,한식,"""미 ""갈매기살전문",None,20031105,익선동,126.991068861385,37.573007173395,54534440.0,733,...,37.938,19.830,11.116,12.763,45.212,7.570,28.394,서울 종로구 돈화문로11가길 7,한식,한식
3,202304,한식,"""미 ""갈매기살전문",None,20031105,익선동,126.991068861385,37.573007173395,84513865.0,1015,...,43.340,13.873,10.164,12.144,57.957,9.534,15.989,서울 종로구 돈화문로11가길 7,한식,한식
4,202305,한식,"""미 ""갈매기살전문",None,20031105,익선동,126.991068861385,37.573007173395,70911682.0,850,...,38.152,21.813,12.378,7.416,58.094,5.134,16.537,서울 종로구 돈화문로11가길 7,한식,한식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406285,202308,양식,（주）에스알인베,None,20100121,수서역,127.103226986877,37.488510452832,49810547.0,1128,...,22.435,19.344,18.120,28.904,24.578,11.407,39.024,"서울 강남구 광평로51길 6-11,",양식,None
1406286,202309,양식,（주）에스알인베,None,20100121,수서역,127.103226986877,37.488510452832,40096531.0,1015,...,27.881,19.919,12.677,29.676,28.544,13.324,36.674,"서울 강남구 광평로51길 6-11,",양식,None
1406287,202310,양식,（주）에스알인베,None,20100121,수서역,127.103226986877,37.488510452832,33497913.0,874,...,28.690,22.706,17.848,23.248,32.342,16.282,26.179,"서울 강남구 광평로51길 6-11,",양식,None
1406288,202311,양식,（주）에스알인베,None,20100121,수서역,127.103226986877,37.488510452832,29845619.0,853,...,29.292,14.482,12.809,27.874,34.191,20.221,24.117,"서울 강남구 광평로51길 6-11,",양식,None


In [91]:
final_df[final_df['MCTNM']=='화신먹을거리촌']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
1386109,202301,한식,화신먹을거리촌,None,20100803,인사동,126.984830437457,37.571691041759,1373382.0,29,...,38.356,61.644,0.000,0.000,38.356,0.000,61.644,서울 종로구 인사동,한식,별식/퓨전요리
1386110,202302,한식,화신먹을거리촌,None,20100803,인사동,126.984830437457,37.571691041759,1470140.0,28,...,11.111,11.111,0.000,77.778,22.222,0.000,42.593,서울 종로구 인사동,한식,별식/퓨전요리
1386111,202303,한식,화신먹을거리촌,None,20100803,인사동,126.984830437457,37.571691041759,737816.0,17,...,77.778,0.000,22.222,0.000,44.444,0.000,0.000,서울 종로구 인사동,한식,별식/퓨전요리
1386112,202304,한식,화신먹을거리촌,None,20100803,인사동,126.984830437457,37.571691041759,1229209.0,27,...,0.000,0.000,46.256,20.705,33.040,20.705,46.256,서울 종로구 인사동,한식,별식/퓨전요리
1386113,202305,한식,화신먹을거리촌,None,20100803,인사동,126.984830437457,37.571691041759,1088771.0,24,...,75.000,22.222,0.000,2.778,47.222,22.222,30.556,서울 종로구 인사동,한식,별식/퓨전요리
1386114,202306,한식,화신먹을거리촌,None,20100803,인사동,126.984830437457,37.571691041759,2182382.0,39,...,13.462,65.385,21.154,0.000,78.846,0.000,21.154,서울 종로구 인사동,한식,별식/퓨전요리
1386115,202307,한식,화신먹을거리촌,None,20100803,인사동,126.984830437457,37.571691041759,844241.0,24,...,27.536,25.362,0.000,47.101,27.536,28.986,43.478,서울 종로구 인사동,한식,별식/퓨전요리
1386116,202308,한식,화신먹을거리촌,None,20100803,인사동,126.984830437457,37.571691041759,3148724.0,60,...,38.936,12.605,36.415,12.045,31.653,11.204,20.728,서울 종로구 인사동,한식,별식/퓨전요리
1386117,202309,한식,화신먹을거리촌,None,20100803,인사동,126.984830437457,37.571691041759,2137447.0,26,...,100.000,0.000,0.000,0.000,0.000,0.000,0.000,서울 종로구 인사동,한식,별식/퓨전요리
1386118,202310,한식,화신먹을거리촌,None,20100803,인사동,126.984830437457,37.571691041759,1681900.0,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,서울 종로구 인사동,한식,별식/퓨전요리


In [82]:
result_df.to_csv("신한카드_월매출_전처리.csv", index=False, encoding='utf-8-sig')

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM,group_key
718779,202301,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,32082497.0,442,...,20.308,13.501,4.576,68.725,2.912,17.340,서울 종로구 돈화문로11가길 7,한식,한식,NaN
163056,202302,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,51156355.0,704,...,28.429,9.758,7.113,60.367,6.643,20.998,서울 종로구 돈화문로11가길 7,한식,한식,NaN
884033,202303,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,54534440.0,733,...,19.830,11.116,12.763,45.212,7.570,28.394,서울 종로구 돈화문로11가길 7,한식,한식,NaN
584501,202304,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,84513865.0,1015,...,13.873,10.164,12.144,57.957,9.534,15.989,서울 종로구 돈화문로11가길 7,한식,한식,NaN
50193,202305,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,70911682.0,850,...,21.813,12.378,7.416,58.094,5.134,16.537,서울 종로구 돈화문로11가길 7,한식,한식,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380521,202308,양식,（주）에스알인베,NaN,20100121,수서역,127.103226986877,37.488510452832,49810547.0,1128,...,19.344,18.120,28.904,24.578,11.407,39.024,"서울 강남구 광평로51길 6-11,",양식,NaN,NaN
129651,202309,양식,（주）에스알인베,NaN,20100121,수서역,127.103226986877,37.488510452832,40096531.0,1015,...,19.919,12.677,29.676,28.544,13.324,36.674,"서울 강남구 광평로51길 6-11,",양식,NaN,NaN
344382,202310,양식,（주）에스알인베,NaN,20100121,수서역,127.103226986877,37.488510452832,33497913.0,874,...,22.706,17.848,23.248,32.342,16.282,26.179,"서울 강남구 광평로51길 6-11,",양식,NaN,NaN
1147106,202311,양식,（주）에스알인베,NaN,20100121,수서역,127.103226986877,37.488510452832,29845619.0,853,...,14.482,12.809,27.874,34.191,20.221,24.117,"서울 강남구 광평로51길 6-11,",양식,NaN,NaN


In [4]:
df[df['MCTNM']=='힘쎈장어 부끄러운새우']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
1410428,202301,한식,힘쎈장어 부끄러운새우,NaN,20220315,도곡공원,127.039077,37.492962,1069150.0,29,...,56.451,25.926,0.000,17.623,82.377,0.000,17.623,서울특별시 강남구 논현로63길 16,한식,수산물
1410429,202302,한식,힘쎈장어 부끄러운새우,NaN,20220315,도곡공원,127.039077,37.492962,3580736.0,49,...,0.000,62.144,18.466,7.161,39.551,0.000,52.238,서울특별시 강남구 논현로63길 16,한식,수산물
1410430,202303,한식,힘쎈장어 부끄러운새우,NaN,20220315,도곡공원,127.039077,37.492962,2604167.0,49,...,42.008,21.141,0.000,7.391,64.286,7.183,21.557,서울특별시 강남구 논현로63길 16,한식,수산물
1410431,202304,한식,힘쎈장어 부끄러운새우,NaN,20220315,도곡공원,127.039077,37.492962,4817010.0,111,...,68.649,5.551,11.258,9.773,65.991,4.769,29.240,서울특별시 강남구 논현로63길 16,한식,수산물
1410432,202305,한식,힘쎈장어 부끄러운새우,NaN,20220315,도곡공원,127.039077,37.492962,3467175.0,61,...,8.071,0.000,7.758,26.289,49.728,0.000,34.937,서울특별시 강남구 논현로63길 16,한식,수산물
1410433,202306,한식,힘쎈장어 부끄러운새우,NaN,20220315,도곡공원,127.039077,37.492962,2585383.5,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,서울특별시 강남구 논현로63길 16,한식,수산물
1410434,202307,한식,힘쎈장어 부끄러운새우,NaN,20220315,도곡공원,127.039077,37.492962,1703592.0,21,...,100.000,0.000,0.000,0.000,100.000,0.000,0.000,서울특별시 강남구 논현로63길 16,한식,수산물
1410435,202308,한식,힘쎈장어 부끄러운새우,NaN,20220315,도곡공원,127.039077,37.492962,1436657.0,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,서울특별시 강남구 논현로63길 16,한식,수산물
1410436,202309,한식,힘쎈장어 부끄러운새우,NaN,20220315,도곡공원,127.039077,37.492962,1169722.0,29,...,0.000,0.000,0.000,0.000,100.000,0.000,0.000,서울특별시 강남구 논현로63길 16,한식,수산물


In [6]:
df[df['MCTNM']=='힐스(Hillw\'s) 예찬']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
1410142,202302,식품잡화,힐스(Hillw's) 예찬,NaN,20210907,신정1동,126.856367,37.52033,664962.0,20,...,47.0,0.000,0.0,53.0,47.000,0.0,53.000,"서울 양천구 중앙로36길 15,",식품,소매
1410143,202303,식품잡화,힐스(Hillw's) 예찬,NaN,20210907,신정1동,126.856367,37.52033,491343.5,0,...,0.0,0.000,0.0,0.0,0.000,0.0,0.000,"서울 양천구 중앙로36길 15,",식품,소매
1410144,202304,식품잡화,힐스(Hillw's) 예찬,NaN,20210907,신정1동,126.856367,37.52033,317725.0,16,...,100.0,0.000,0.0,0.0,100.000,0.0,0.000,"서울 양천구 중앙로36길 15,",식품,소매
1410145,202305,식품잡화,힐스(Hillw's) 예찬,NaN,20210907,신정1동,126.856367,37.52033,149086.0,15,...,0.0,48.274,0.0,0.0,51.726,0.0,48.274,"서울 양천구 중앙로36길 15,",식품,소매


In [7]:
df[df['MCTNM']=='힐사이드라이브']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
1410108,202301,일반대중음식,힐사이드라이브,NaN,20171019,자양,127.083841,37.535136,1653551.0,18,...,81.618,0.0,0.0,0.000,55.147,0.00,0.000,서울특별시 광진구 자양로 81,한식,NaN
1410109,202302,일반대중음식,힐사이드라이브,NaN,20171019,자양,127.083841,37.535136,5064038.0,0,...,0.000,0.0,0.0,0.000,0.000,0.00,0.000,서울특별시 광진구 자양로 81,한식,NaN
1410110,202303,일반대중음식,힐사이드라이브,NaN,20171019,자양,127.083841,37.535136,8474525.0,28,...,0.000,0.0,0.0,100.000,0.000,0.00,0.000,서울특별시 광진구 자양로 81,한식,NaN
1410111,202304,일반대중음식,힐사이드라이브,NaN,20171019,자양,127.083841,37.535136,2020971.0,16,...,32.990,0.0,0.0,67.010,0.000,67.01,20.103,서울특별시 광진구 자양로 81,한식,NaN
1410112,202305,일반대중음식,힐사이드라이브,NaN,20171019,자양,127.083841,37.535136,4437103.5,0,...,0.000,0.0,0.0,0.000,0.000,0.00,0.000,서울특별시 광진구 자양로 81,한식,NaN
1410113,202306,일반대중음식,힐사이드라이브,NaN,20171019,자양,127.083841,37.535136,6853236.0,29,...,0.000,0.0,0.0,0.000,0.000,0.00,0.000,서울특별시 광진구 자양로 81,한식,NaN
1410114,202307,일반대중음식,힐사이드라이브,NaN,20171019,자양,127.083841,37.535136,6300854.0,16,...,10.000,90.0,0.0,0.000,0.000,0.00,0.000,서울특별시 광진구 자양로 81,한식,NaN
1410115,202308,일반대중음식,힐사이드라이브,NaN,20171019,자양,127.083841,37.535136,7094753.5,0,...,0.000,0.0,0.0,0.000,0.000,0.00,0.000,서울특별시 광진구 자양로 81,한식,NaN
1410116,202309,일반대중음식,힐사이드라이브,NaN,20171019,자양,127.083841,37.535136,7094753.5,0,...,0.000,0.0,0.0,0.000,0.000,0.00,0.000,서울특별시 광진구 자양로 81,한식,NaN
1410117,202310,일반대중음식,힐사이드라이브,NaN,20171019,자양,127.083841,37.535136,7094753.5,0,...,0.000,0.0,0.0,0.000,0.000,0.00,0.000,서울특별시 광진구 자양로 81,한식,NaN


In [8]:
df[df['MCTNM']=='히토사라']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
1409552,202301,일식,히토사라,NaN,20211028,NaN,127.080765,37.620639,20925288.0,141,...,16.682,43.505,8.044,19.374,58.748,10.110,15.430,서울특별시 노원구 노원로1바길 2,아시아음식,NaN
1409553,202302,일식,히토사라,NaN,20211028,NaN,127.080765,37.620639,28258478.0,167,...,42.916,2.180,25.599,14.313,51.393,11.577,28.554,서울특별시 노원구 노원로1바길 2,아시아음식,NaN
1409554,202303,일식,히토사라,NaN,20211028,NaN,127.080765,37.620639,29051880.0,206,...,33.978,19.580,8.986,17.357,54.598,18.374,27.028,서울특별시 노원구 노원로1바길 2,아시아음식,NaN
1409555,202304,일식,히토사라,NaN,20211028,NaN,127.080765,37.620639,20029274.0,135,...,49.902,10.719,11.811,13.854,58.998,0.000,11.783,서울특별시 노원구 노원로1바길 2,아시아음식,NaN
1409556,202305,일식,히토사라,NaN,20211028,NaN,127.080765,37.620639,23340391.0,146,...,40.490,35.491,7.246,7.380,62.965,7.716,24.187,서울특별시 노원구 노원로1바길 2,아시아음식,NaN
1409557,202306,일식,히토사라,NaN,20211028,NaN,127.080765,37.620639,14783864.0,96,...,35.312,19.854,10.023,19.275,60.871,3.855,20.046,서울특별시 노원구 노원로1바길 2,아시아음식,NaN
1409558,202307,일식,히토사라,NaN,20211028,NaN,127.080765,37.620639,10010974.0,52,...,47.700,18.160,18.160,0.000,36.804,26.877,36.320,서울특별시 노원구 노원로1바길 2,아시아음식,NaN
1409559,202308,일식,히토사라,NaN,20211028,NaN,127.080765,37.620639,8631403.5,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,서울특별시 노원구 노원로1바길 2,아시아음식,NaN
1409560,202309,일식,히토사라,NaN,20211028,NaN,127.080765,37.620639,7251833.0,31,...,44.396,32.460,23.144,0.000,80.640,0.000,0.000,서울특별시 노원구 노원로1바길 2,아시아음식,NaN
1409561,202310,일식,히토사라,NaN,20211028,NaN,127.080765,37.620639,15371725.0,105,...,36.055,7.460,30.584,14.546,47.410,0.000,25.736,서울특별시 노원구 노원로1바길 2,아시아음식,NaN


In [9]:
df[df['MCTNM']=='히카루']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
1409350,202301,일반대중음식,히카루,NaN,20150514,보라매공원,126.926923,37.491158,21499792.0,447,...,40.126,25.517,9.460,10.130,57.403,8.970,19.072,"서울 관악구 보라매로 17,",주점및주류판매,유흥주점
1409351,202302,일반대중음식,히카루,NaN,20150514,보라매공원,126.926923,37.491158,14848817.0,313,...,60.831,13.160,7.180,6.438,74.747,5.873,12.768,"서울 관악구 보라매로 17,",주점및주류판매,유흥주점
1409352,202303,일반대중음식,히카루,NaN,20150514,보라매공원,126.926923,37.491158,22658905.0,381,...,55.042,21.852,0.847,4.392,69.474,4.439,13.554,"서울 관악구 보라매로 17,",주점및주류판매,유흥주점
1409353,202304,일반대중음식,히카루,NaN,20150514,보라매공원,126.926923,37.491158,16804533.0,329,...,46.403,28.513,4.619,10.020,50.329,8.492,30.627,"서울 관악구 보라매로 17,",주점및주류판매,유흥주점
1409354,202312,일식,히카루,NaN,20231106,상암동MBC,126.896611,37.576793,12166832.0,155,...,84.860,8.643,3.406,0.000,76.302,5.094,4.980,서울특별시 마포구 성암로15길 47,아시아음식,NaN


In [10]:
df2 = pd.read_csv('month_fill.csv', encoding='utf-8-sig')

In [11]:
df2[df2['MCTNM']=='히카루']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,여성_60대,소득정보없음,소득3천만이하,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년
1405051,202301,일반대중음식,히카루,NaN,20150514,보라매공원,126.926923,37.491158,21499792,447,...,0.000,0.0,14.768,40.126,25.517,9.460,10.130,57.403,8.970,19.072
1405052,202302,일반대중음식,히카루,NaN,20150514,보라매공원,126.926923,37.491158,14848817,313,...,0.000,0.0,12.391,60.831,13.160,7.180,6.438,74.747,5.873,12.768
1405053,202303,일반대중음식,히카루,NaN,20150514,보라매공원,126.926923,37.491158,22658905,381,...,0.000,0.0,17.866,55.042,21.852,0.847,4.392,69.474,4.439,13.554
1405054,202304,일반대중음식,히카루,NaN,20150514,보라매공원,126.926923,37.491158,16804533,329,...,0.000,0.0,10.446,46.403,28.513,4.619,10.020,50.329,8.492,30.627
1405055,202305,일반대중음식,히카루,NaN,20150514,보라매공원,126.926923,37.491158,0,0,...,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1405056,202306,일반대중음식,히카루,NaN,20150514,보라매공원,126.926923,37.491158,0,0,...,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1405057,202307,일반대중음식,히카루,NaN,20150514,보라매공원,126.926923,37.491158,0,0,...,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1405058,202308,일반대중음식,히카루,NaN,20150514,보라매공원,126.926923,37.491158,0,0,...,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1405059,202309,일반대중음식,히카루,NaN,20150514,보라매공원,126.926923,37.491158,0,0,...,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1405060,202310,일반대중음식,히카루,NaN,20150514,보라매공원,126.926923,37.491158,0,0,...,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [17]:
df[df['MCTNM']=='히어로']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
1408827,202301,한식,히어로,NaN,20211202,강남역,127.032841,37.491614,13269399.0,24,...,0.000,0.000,0.000,100.000,100.000,0.000,0.000,서울특별시 강남구 도곡로3길 22,한식,패스트푸드
1408828,202302,한식,히어로,NaN,20211202,강남역,127.032841,37.491614,10767435.0,26,...,0.000,0.000,0.000,100.000,70.000,0.000,30.000,서울특별시 강남구 도곡로3길 22,한식,패스트푸드
1408829,202303,한식,히어로,NaN,20211202,강남역,127.032841,37.491614,17196636.0,32,...,0.000,6.122,0.000,93.878,6.122,0.000,93.878,서울특별시 강남구 도곡로3길 22,한식,패스트푸드
1408830,202304,한식,히어로,NaN,20211202,강남역,127.032841,37.491614,16100057.0,21,...,0.000,0.000,0.000,69.751,69.751,0.000,30.249,서울특별시 강남구 도곡로3길 22,한식,패스트푸드
1408831,202305,한식,히어로,NaN,20211202,강남역,127.032841,37.491614,14560528.0,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,서울특별시 강남구 도곡로3길 22,한식,패스트푸드
1408832,202306,한식,히어로,NaN,20211202,강남역,127.032841,37.491614,13020999.0,16,...,0.000,38.839,0.000,61.161,38.839,0.000,61.161,서울특별시 강남구 도곡로3길 22,한식,패스트푸드
1408833,202307,한식,히어로,NaN,20211202,강남역,127.032841,37.491614,14161947.0,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,서울특별시 강남구 도곡로3길 22,한식,패스트푸드
1408834,202308,한식,히어로,NaN,20211202,강남역,127.032841,37.491614,14161947.0,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,서울특별시 강남구 도곡로3길 22,한식,패스트푸드
1408835,202309,한식,히어로,NaN,20211202,강남역,127.032841,37.491614,15302895.0,16,...,0.000,0.000,68.668,31.332,68.668,0.000,14.424,서울특별시 강남구 도곡로3길 22,한식,패스트푸드
1408836,202310,한식,히어로,NaN,20211202,강남역,127.032841,37.491614,15550998.5,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,서울특별시 강남구 도곡로3길 22,한식,패스트푸드


In [18]:
df[df['MCTNM']=='히비야']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
1408703,202301,한식,히비야,NaN,20140513,경희회기,127.053172,37.592786,2987850.0,126,...,31.617,0.000,16.903,3.008,72.955,8.020,19.025,"서울 동대문구 회기로21길 29,",한식,일식
1408704,202302,한식,히비야,NaN,20140513,경희회기,127.053172,37.592786,713918.0,40,...,31.897,12.061,0.000,0.000,87.939,0.000,0.000,"서울 동대문구 회기로21길 29,",한식,일식
1408705,202303,한식,히비야,NaN,20140513,경희회기,127.053172,37.592786,1494938.0,61,...,44.641,0.000,29.357,0.000,59.460,0.000,40.540,"서울 동대문구 회기로21길 29,",한식,일식
1408706,202304,한식,히비야,NaN,20140513,경희회기,127.053172,37.592786,507931.0,27,...,42.532,23.051,16.465,0.000,43.732,0.000,16.465,"서울 동대문구 회기로21길 29,",한식,일식
1408707,202305,한식,히비야,NaN,20140513,경희회기,127.053172,37.592786,499179.5,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,"서울 동대문구 회기로21길 29,",한식,일식
1408708,202306,한식,히비야,NaN,20140513,경희회기,127.053172,37.592786,490428.0,21,...,75.723,0.000,24.277,0.000,58.618,17.106,0.000,"서울 동대문구 회기로21길 29,",한식,일식
1408709,202304,일식,히비야,NaN,20140509,NaN,126.963201,37.559683,286478.0,21,...,50.909,0.000,0.000,0.000,74.545,0.000,0.000,"서울 서대문구 충정로 8,",아시아음식,일식
1408710,202309,일식,히비야,NaN,20230830,NaN,126.963201,37.559683,15335874.0,773,...,38.839,25.853,9.587,11.373,57.088,6.281,22.811,서울특별시 서대문구 충정로 8,아시아음식,NaN
1408711,202310,일식,히비야,NaN,20230830,NaN,126.963201,37.559683,15845311.0,963,...,29.789,22.923,17.048,8.655,44.975,6.032,32.522,서울특별시 서대문구 충정로 8,아시아음식,NaN
1408712,202311,일식,히비야,NaN,20230830,NaN,126.963201,37.559683,19536300.0,1104,...,31.665,25.575,14.771,9.417,56.953,6.884,20.316,서울특별시 서대문구 충정로 8,아시아음식,NaN


In [19]:
df[df['MCTNM']=='히로']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
1408477,202301,일식,히로,NaN,20150924,남성역,126.969238,37.48539,7609495.0,172,...,51.748,10.793,2.887,11.628,69.479,2.887,20.626,"서울 동작구 사당로 188,",아시아음식,유흥주점
1408478,202302,일식,히로,NaN,20150924,남성역,126.969238,37.48539,8539247.0,151,...,45.113,11.039,18.527,0.000,73.908,0.000,18.913,"서울 동작구 사당로 188,",아시아음식,유흥주점
1408479,202303,일식,히로,NaN,20150924,남성역,126.969238,37.48539,7406144.0,161,...,42.328,3.268,9.151,3.826,71.000,11.144,16.660,"서울 동작구 사당로 188,",아시아음식,유흥주점
1408480,202304,일식,히로,NaN,20150924,남성역,126.969238,37.48539,6332505.0,106,...,32.889,21.574,10.745,0.000,72.610,27.390,0.000,"서울 동작구 사당로 188,",아시아음식,유흥주점
1408481,202306,일식,히로,NaN,20150924,연남동,126.926277,37.56384,8339012.0,109,...,48.025,0.000,21.959,9.637,72.433,0.000,16.667,"서울 마포구 동교로 276,",아시아음식,유흥주점
1408482,202307,일식,히로,NaN,20150924,연남동,126.926277,37.56384,19984937.0,293,...,46.561,10.007,2.212,7.549,84.130,0.000,12.956,"서울 마포구 동교로 276,",아시아음식,유흥주점
1408483,202308,일식,히로,NaN,20150924,연남동,126.926277,37.56384,20780956.0,233,...,40.734,26.915,8.742,5.534,65.096,4.551,24.034,"서울 마포구 동교로 276,",아시아음식,유흥주점
1408484,202309,일식,히로,NaN,20150924,연남동,126.926277,37.56384,12198190.0,175,...,47.110,4.908,7.685,2.131,95.609,0.000,4.391,"서울 마포구 동교로 276,",아시아음식,유흥주점
1408485,202310,일식,히로,NaN,20150924,연남동,126.926277,37.56384,15287212.0,234,...,46.544,12.962,0.000,8.539,71.645,8.482,11.794,"서울 마포구 동교로 276,",아시아음식,유흥주점
1408486,202311,일식,히로,NaN,20150924,연남동,126.926277,37.56384,18467911.0,211,...,60.461,4.754,6.735,13.987,74.555,0.000,18.832,"서울 마포구 동교로 276,",아시아음식,유흥주점


In [30]:
df[df['MCTNM']=='화신먹을거리촌']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
1390502,202301,한식,화신먹을거리촌,NaN,20100803,인사동,126.98483,37.571691,1373382.0,29,...,38.356,61.644,0.000,0.000,38.356,0.000,61.644,서울 종로구 인사동,한식,별식/퓨전요리
1390503,202302,한식,화신먹을거리촌,NaN,20100803,인사동,126.98483,37.571691,1470140.0,28,...,11.111,11.111,0.000,77.778,22.222,0.000,42.593,서울 종로구 인사동,한식,별식/퓨전요리
1390504,202302,한식,화신먹을거리촌,NaN,20100803,인사동,126.98483,37.571691,1492572.0,42,...,31.034,37.931,0.000,0.000,48.276,0.000,31.034,서울 종로구 인사동,한식,별식/퓨전요리
1390505,202303,한식,화신먹을거리촌,NaN,20100803,인사동,126.98483,37.571691,737816.0,17,...,77.778,0.000,22.222,0.000,44.444,0.000,0.000,서울 종로구 인사동,한식,별식/퓨전요리
1390506,202303,한식,화신먹을거리촌,NaN,20100803,인사동,126.98483,37.571691,2920071.0,60,...,21.847,34.910,0.000,11.261,71.171,11.261,0.000,서울 종로구 인사동,한식,별식/퓨전요리
1390507,202304,한식,화신먹을거리촌,NaN,20100803,인사동,126.98483,37.571691,1229209.0,27,...,0.000,0.000,46.256,20.705,33.040,20.705,46.256,서울 종로구 인사동,한식,별식/퓨전요리
1390508,202304,한식,화신먹을거리촌,NaN,20100803,인사동,126.98483,37.571691,1552403.0,33,...,13.393,30.357,56.250,0.000,0.000,33.929,30.357,서울 종로구 인사동,한식,별식/퓨전요리
1390509,202305,한식,화신먹을거리촌,NaN,20100803,인사동,126.98483,37.571691,1088771.0,24,...,75.000,22.222,0.000,2.778,47.222,22.222,30.556,서울 종로구 인사동,한식,별식/퓨전요리
1390510,202305,한식,화신먹을거리촌,NaN,20100803,인사동,126.98483,37.571691,1125004.0,23,...,0.000,0.000,47.794,52.206,0.000,0.000,100.000,서울 종로구 인사동,한식,별식/퓨전요리
1390511,202306,한식,화신먹을거리촌,NaN,20100803,인사동,126.98483,37.571691,2182382.0,39,...,13.462,65.385,21.154,0.000,78.846,0.000,21.154,서울 종로구 인사동,한식,별식/퓨전요리


In [51]:
df[df['MCTNM']=='홍자씨샌드위치 파크리오점']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
1384730,202301,제과점,홍자씨샌드위치 파크리오점,홍자씨샌드위치,20220816,NaN,127.110349,37.521437,656124.0,34,...,36.215,0.000,0.000,32.843,67.157,0.000,32.843,서울특별시 송파구 올림픽로37길 130,간식,패스트푸드
1384731,202302,제과점,홍자씨샌드위치 파크리오점,홍자씨샌드위치,20220816,NaN,127.110349,37.521437,488312.5,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,서울특별시 송파구 올림픽로37길 130,간식,패스트푸드
1384732,202303,제과점,홍자씨샌드위치 파크리오점,홍자씨샌드위치,20220816,NaN,127.110349,37.521437,320501.0,16,...,42.166,0.000,0.000,57.834,0.000,0.000,69.940,서울특별시 송파구 올림픽로37길 130,간식,패스트푸드
1384733,202304,제과점,홍자씨샌드위치 파크리오점,홍자씨샌드위치,20220816,NaN,127.110349,37.521437,713984.5,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,서울특별시 송파구 올림픽로37길 130,간식,패스트푸드
1384734,202305,제과점,홍자씨샌드위치 파크리오점,홍자씨샌드위치,20220816,NaN,127.110349,37.521437,713984.5,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,서울특별시 송파구 올림픽로37길 130,간식,패스트푸드
1384735,202306,제과점,홍자씨샌드위치 파크리오점,홍자씨샌드위치,20220816,NaN,127.110349,37.521437,713984.5,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,서울특별시 송파구 올림픽로37길 130,간식,패스트푸드
1384736,202307,제과점,홍자씨샌드위치 파크리오점,홍자씨샌드위치,20220816,NaN,127.110349,37.521437,1107468.0,23,...,26.051,20.979,52.970,0.000,0.000,12.321,61.628,서울특별시 송파구 올림픽로37길 130,간식,패스트푸드
1384737,202308,제과점,홍자씨샌드위치 파크리오점,홍자씨샌드위치,20220816,NaN,127.110349,37.521437,476318.0,21,...,46.391,0.000,0.000,53.609,27.420,28.428,0.000,서울특별시 송파구 올림픽로37길 130,간식,패스트푸드
1384738,202309,제과점,홍자씨샌드위치 파크리오점,홍자씨샌드위치,20220816,NaN,127.110349,37.521437,976737.0,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,서울특별시 송파구 올림픽로37길 130,간식,패스트푸드
1384739,202310,제과점,홍자씨샌드위치 파크리오점,홍자씨샌드위치,20220816,NaN,127.110349,37.521437,976737.0,0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,서울특별시 송파구 올림픽로37길 130,간식,패스트푸드


In [69]:
df[df['MCTNM']=='흥부네 사랑채']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
1406417,202301,한식,흥부네 사랑채,NaN,20101116,NaN,0.0,0.0,5134359.0,321,...,53.013,6.138,12.277,8.929,18.973,11.384,9.263,서울 종로구 난계로29길 53,한식,NaN
1406418,202302,한식,흥부네 사랑채,NaN,20101116,NaN,0.0,0.0,5903145.0,361,...,51.426,9.980,3.666,16.904,16.293,2.138,19.246,서울 종로구 난계로29길 53,한식,NaN
1406419,202303,한식,흥부네 사랑채,NaN,20101116,NaN,0.0,0.0,4832077.0,333,...,51.020,14.166,3.842,18.247,22.569,3.001,26.651,서울 종로구 난계로29길 53,한식,NaN
1406420,202304,한식,흥부네 사랑채,NaN,20101116,NaN,0.0,0.0,5474004.0,393,...,40.000,10.918,7.857,15.306,27.857,0.000,33.571,서울 종로구 난계로29길 53,한식,NaN
1406421,202305,한식,흥부네 사랑채,NaN,20101116,NaN,0.0,0.0,10457964.0,448,...,43.887,2.926,7.941,5.329,33.229,0.836,15.361,서울 종로구 난계로29길 53,한식,NaN
1406422,202306,한식,흥부네 사랑채,NaN,20101116,NaN,0.0,0.0,12172172.0,562,...,24.223,17.256,28.617,6.109,6.645,22.079,14.362,서울 종로구 난계로29길 53,한식,NaN
1406423,202307,한식,흥부네 사랑채,NaN,20101116,NaN,0.0,0.0,21406675.0,351,...,10.453,4.643,2.697,79.566,3.003,0.000,4.003,서울 종로구 난계로29길 53,한식,NaN
1406424,202308,한식,흥부네 사랑채,NaN,20101116,NaN,0.0,0.0,5527755.0,343,...,48.408,27.003,5.159,2.854,21.954,0.000,9.001,서울 종로구 난계로29길 53,한식,NaN
1406425,202309,한식,흥부네 사랑채,NaN,20101116,NaN,0.0,0.0,4606096.0,281,...,56.195,14.643,7.509,0.000,32.290,7.885,10.638,서울 종로구 난계로29길 53,한식,NaN
1406426,202310,한식,흥부네 사랑채,NaN,20101116,NaN,0.0,0.0,8128273.0,414,...,43.227,10.956,10.159,14.542,15.538,2.390,17.331,서울 종로구 난계로29길 53,한식,NaN


In [53]:
df[df['LALXCVL']==0]

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
737,202301,한식,(우리)풍천장어,풍천장어,20031121,NaN,0.0,0.0,19223094.0,316,...,40.711,3.557,25.081,12.684,15.056,4.024,43.838,서울 성북구 장월로 134-1,한식,수산물
738,202302,한식,(우리)풍천장어,풍천장어,20031121,NaN,0.0,0.0,18621115.0,303,...,46.241,17.275,13.624,9.758,25.130,2.792,34.244,서울 성북구 장월로 134-1,한식,수산물
739,202303,한식,(우리)풍천장어,풍천장어,20031121,NaN,0.0,0.0,32507694.0,535,...,36.764,21.092,13.195,12.343,13.987,3.451,16.646,서울 성북구 장월로 134-1,한식,수산물
740,202304,한식,(우리)풍천장어,풍천장어,20031121,NaN,0.0,0.0,22151892.0,325,...,39.346,15.497,24.293,5.340,6.702,1.466,33.534,서울 성북구 장월로 134-1,한식,수산물
741,202305,한식,(우리)풍천장어,풍천장어,20031121,NaN,0.0,0.0,34068820.0,432,...,41.691,21.447,10.337,3.912,29.684,12.473,17.103,서울 성북구 장월로 134-1,한식,수산물
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406424,202308,한식,흥부네 사랑채,NaN,20101116,NaN,0.0,0.0,5527755.0,343,...,48.408,27.003,5.159,2.854,21.954,0.000,9.001,서울 종로구 난계로29길 53,한식,NaN
1406425,202309,한식,흥부네 사랑채,NaN,20101116,NaN,0.0,0.0,4606096.0,281,...,56.195,14.643,7.509,0.000,32.290,7.885,10.638,서울 종로구 난계로29길 53,한식,NaN
1406426,202310,한식,흥부네 사랑채,NaN,20101116,NaN,0.0,0.0,8128273.0,414,...,43.227,10.956,10.159,14.542,15.538,2.390,17.331,서울 종로구 난계로29길 53,한식,NaN
1406427,202311,한식,흥부네 사랑채,NaN,20101116,NaN,0.0,0.0,6326188.0,413,...,54.589,14.779,7.151,6.436,13.111,6.317,9.774,서울 종로구 난계로29길 53,한식,NaN


In [47]:
data_sorted = df.sort_values(by=['MCTNM', 'LALXCVL', 'LALYCVL', 'TA_YM'])

# 같은 매장명+(위도, 경도) 조합에서 같은 월에 데이터가 2개 이상인 경우만 필터링
# groupby를 사용하여 각 그룹별로 카운트를 계산하고, 카운트가 2 이상인 경우만 선택
filtered = data_sorted.groupby(['MCTNM', 'LALXCVL', 'LALYCVL', 'TA_YM']).filter(lambda x: len(x) > 1)

filtered

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
1422,202308,농수산물,(유)장성수산,NaN,20121015,가락시장,127.111147,37.493479,2991116.0,94,...,31.218,25.934,20.049,10.899,38.508,0.000,54.508,서울 송파구 양재대로,농축수산물,수산물
1427,202308,농수산물,(유)장성수산,NaN,20201230,가락시장,127.111147,37.493479,3459540.0,89,...,11.322,2.772,39.281,16.909,53.845,15.982,27.401,서울특별시 송파구 양재대로 932,농축수산물,수산물
1423,202309,농수산물,(유)장성수산,NaN,20121015,가락시장,127.111147,37.493479,8303737.0,157,...,39.031,14.251,1.825,30.902,51.525,24.373,16.251,서울 송파구 양재대로,농축수산물,수산물
1428,202309,농수산물,(유)장성수산,NaN,20201230,가락시장,127.111147,37.493479,1982850.0,62,...,58.819,0.000,4.383,21.805,63.496,8.385,5.280,서울특별시 송파구 양재대로 932,농축수산물,수산물
1424,202310,농수산물,(유)장성수산,NaN,20121015,가락시장,127.111147,37.493479,13588539.0,335,...,45.699,26.425,9.251,8.184,66.248,6.524,15.836,서울 송파구 양재대로,농축수산물,수산물
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1401443,202303,일반대중음식,후라이드참잘하는집 왕십리행당점,후라이드참잘하는집,20190124,왕십리,127.034348,37.565107,34752748.0,1849,...,34.678,7.591,6.945,11.258,69.194,6.032,15.203,서울특별시 성동구 왕십리로24길 27-1,한식,별식/퓨전요리
1408201,202312,한식,히노하루,NaN,20191203,경희회기,127.056729,37.591488,30307806.0,251,...,56.390,15.476,15.160,4.300,71.963,7.920,5.175,서울특별시 동대문구 이문로 45,아시아음식,일식
1408202,202312,일식,히노하루,NaN,20231107,경희회기,127.056729,37.591488,1262391.0,35,...,59.573,14.052,0.000,23.192,62.756,0.000,0.000,서울특별시 동대문구 이문로 45,아시아음식,NaN
1410276,202301,패스트푸드,힘난다버거 상암점,힘난다버거,20211214,상암동MBC,126.894252,37.578582,461878.0,17,...,20.343,36.098,0.000,0.000,63.902,0.000,36.098,서울특별시 마포구 매봉산로 80,패스트푸드,패스트푸드


## 이전 기록

In [7]:
df.sort_values(['MCTNM', 'TA_YM'])

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
718779,202301,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,32082497,442,...,41.902,20.308,13.501,4.576,68.725,2.912,17.340,서울 종로구 돈화문로11가길 7,한식,한식
163056,202302,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,51156355,704,...,34.331,28.429,9.758,7.113,60.367,6.643,20.998,서울 종로구 돈화문로11가길 7,한식,한식
884033,202303,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,54534440,733,...,37.938,19.830,11.116,12.763,45.212,7.570,28.394,서울 종로구 돈화문로11가길 7,한식,한식
584501,202304,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,84513865,1015,...,43.340,13.873,10.164,12.144,57.957,9.534,15.989,서울 종로구 돈화문로11가길 7,한식,한식
50193,202305,한식,"""미 ""갈매기살전문",NaN,20031105,익선동,126.991068861385,37.573007173395,70911682,850,...,38.152,21.813,12.378,7.416,58.094,5.134,16.537,서울 종로구 돈화문로11가길 7,한식,한식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380521,202308,양식,（주）에스알인베,NaN,20100121,수서역,127.103226986877,37.488510452832,49810547,1128,...,22.435,19.344,18.120,28.904,24.578,11.407,39.024,"서울 강남구 광평로51길 6-11,",양식,NaN
129651,202309,양식,（주）에스알인베,NaN,20100121,수서역,127.103226986877,37.488510452832,40096531,1015,...,27.881,19.919,12.677,29.676,28.544,13.324,36.674,"서울 강남구 광평로51길 6-11,",양식,NaN
344382,202310,양식,（주）에스알인베,NaN,20100121,수서역,127.103226986877,37.488510452832,33497913,874,...,28.690,22.706,17.848,23.248,32.342,16.282,26.179,"서울 강남구 광평로51길 6-11,",양식,NaN
1147106,202311,양식,（주）에스알인베,NaN,20100121,수서역,127.103226986877,37.488510452832,29845619,853,...,29.292,14.482,12.809,27.874,34.191,20.221,24.117,"서울 강남구 광평로51길 6-11,",양식,NaN


In [8]:
df[df['BRD_NM']=='스타벅스']

,TA_YM,MCT_RY_NM,MCTNM,BRD_NM,ARED,BZNCD,LALXCVL,LALYCVL,EST_AMT_TOT,EST_CNT_TOT,...,소득5천만이하,소득7천만이하,소득1억이하,소득1억초과,생애단계_싱글,생애단계_영유아,생애단계_청소년,MCT_BSE_AR,MCT_RY_NM2,OUB_RY_MI_CZ_NM
30699,202303,커피전문점,주식회사 에스씨케이컴퍼니,스타벅스,20020919,명동,126.982948495541,37.560193082688,71666246214,7680788,...,35.293,18.553,11.508,12.172,44.158,12.210,26.079,"서울 중구 퇴계로 100,",음료,전자/정보통신
49735,202308,커피전문점,주식회사 에스씨케이컴퍼니,스타벅스,20020919,명동,126.982948495541,37.560193082688,70506743548,7594440,...,35.173,19.120,11.700,12.666,42.855,11.711,26.882,"서울 중구 퇴계로 100,",음료,전자/정보통신
121532,202304,커피전문점,주식회사 에스씨케이컴퍼니,스타벅스,20020919,명동,126.982948495541,37.560193082688,68934482731,7558701,...,35.283,18.727,11.533,12.202,44.077,12.088,26.261,"서울 중구 퇴계로 100,",음료,전자/정보통신
351196,202311,커피전문점,주식회사 에스씨케이컴퍼니,스타벅스,20020919,명동,126.982948495541,37.560193082688,73518464691,7302995,...,35.130,20.094,11.991,12.699,41.589,11.801,27.321,"서울 중구 퇴계로 100,",음료,전자/정보통신
431891,202309,커피전문점,주식회사 에스씨케이컴퍼니,스타벅스,20020919,명동,126.982948495541,37.560193082688,72214287003,7524387,...,35.374,19.556,11.675,12.494,42.680,12.214,26.749,"서울 중구 퇴계로 100,",음료,전자/정보통신
456250,202305,커피전문점,주식회사 에스씨케이컴퍼니,스타벅스,20020919,명동,126.982948495541,37.560193082688,72825699360,7786613,...,35.120,19.114,11.501,12.376,43.100,12.333,26.585,"서울 중구 퇴계로 100,",음료,전자/정보통신
588208,202306,커피전문점,주식회사 에스씨케이컴퍼니,스타벅스,20020919,명동,126.982948495541,37.560193082688,71448228995,7571325,...,34.807,19.345,11.646,12.633,42.972,11.789,26.884,"서울 중구 퇴계로 100,",음료,전자/정보통신
670655,202312,커피전문점,주식회사 에스씨케이컴퍼니,스타벅스,20020919,명동,126.982948495541,37.560193082688,74207392993,7268534,...,34.917,20.605,12.228,13.000,40.327,11.763,27.879,"서울 중구 퇴계로 100,",음료,전자/정보통신
763631,202301,커피전문점,주식회사 에스씨케이컴퍼니,스타벅스,20020919,명동,126.982948495541,37.560193082688,66075277974,7173461,...,36.588,17.442,11.474,11.976,44.338,12.344,26.309,"서울 중구 퇴계로 100,",음료,전자/정보통신
839390,202307,커피전문점,주식회사 에스씨케이컴퍼니,스타벅스,20020919,명동,126.982948495541,37.560193082688,71910410151,7671048,...,35.147,19.124,11.620,12.569,42.969,12.039,26.792,"서울 중구 퇴계로 100,",음료,전자/정보통신
